<a href="https://colab.research.google.com/github/YounSooKimTech/NLP_Power/blob/main/Enron_preprocessing_N_tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings("ignore")

! pip install opendatasets
import opendatasets as od
import pandas
import re  

od.download('https://www.kaggle.com/datasets/wcukierski/enron-email-dataset')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: kim4321
Your Kaggle Key: ··········


100%|██████████| 358M/358M [00:03<00:00, 94.9MB/s]


KeyboardInterrupt: ignored

# Get the dataset

In [ ]:
import pandas as pd

df = pd.read_csv('/content/enron-email-dataset/emails.csv')


In [ ]:
# use the library to parse the dataset
import email
import pandas as pd
from email import utils

def extract_email_info(email_string):
    email_message = email.message_from_string(email_string)
    sender = email_message['From']
    receiver = email_message["To"]
    n_receivers = len(utils.getaddresses([email_message['To']]))
    CC = email_message["CC"]
    subject = email_message['Subject']
    date = email_message['Date']
    # Extract email body
    body = ""
    if email_message.is_multipart():
        for part in email_message.get_payload():
            if part.get_content_type() == 'text/plain':
                body = part.get_payload()
                break
    else:
        body = email_message.get_payload()
    return sender, receiver, n_receivers, CC, subject, body, date

df[['Sender',"Receiver", 'n_receivers', 'CC', 'Subject', 'Body', "Date"]] = df['message'].apply(extract_email_info).apply(pd.Series)

In [ ]:
df.to_csv("parsed_Enron_df.csv", index = False)

from google.colab import files
files.download('parsed_Enron_df.csv') 

In [ ]:
# got the domain for the emails
df['Sender_domain'] = df['Sender'].str.split('@').str[1]
df['Receiver_domain'] = df['Receiver'].str.split('@').str[1]

# 1st filter: n_receiver=1, n_sender=1, no CC, use enron.com
 
filtered_df = df[(df["n_receivers"]== 1) & (df.CC.isna()) & (df.Sender_domain	== "enron.com") & (df.Receiver_domain	== "enron.com")]

# additional filtering based on value_counts
filtered_df = filtered_df[(filtered_df.Sender != "enron.announcements@enron.com") & (filtered_df.Receiver != "all.worldwide@enron.com") ]


In [ ]:
# got the domain for the emails
df['Sender_domain'] = df['Sender'].str.split('@').str[1]
df['Receiver_domain'] = df['Receiver'].str.split('@').str[1]

# 1st filter: n_receiver=1, n_sender=1, no CC, use enron.com
 
filtered_df = df[(df["n_receivers"]== 1) & (df.CC.isna()) & (df.Sender_domain	== "enron.com") & (df.Receiver_domain	== "enron.com")]

# additional filtering based on value_counts
filtered_df = filtered_df[(filtered_df.Sender != "enron.announcements@enron.com") & (filtered_df.Receiver != "all.worldwide@enron.com") ]


In [ ]:
# remove the original mail for reply and forwarded by

filtered_df['content'] = filtered_df['Body'].str.split("---------------------- Forwarded by").str[0] 
filtered_df["content"] = filtered_df["content"].str.split("----- Forwarded by").str[0]
filtered_df["content"] = filtered_df["content"].str.split("-----Original").str[0]

filtered_df["content"] = filtered_df.content.replace("\n", "")
filtered_df["content"] = filtered_df["content"].str.split("\n\n\n\n").str[0]

url_pattern = re.compile(r"http\S+|www\S+")
filtered_df['content'] = filtered_df['content'].str.replace(url_pattern, '')


import re

filtered_df['content']= filtered_df["content"].replace(r"Monthly Basis Swap", "",  regex=True)
filtered_df['content'] = filtered_df["content"].replace(r"Daily Basis Swap", "",  regex=True)
filtered_df['content'] = filtered_df["content"].replace(r"Daily for Monthly Index Swap", "",  regex=True)
filtered_df['content'] = filtered_df["content"].replace(r"Fixed Price Physical", "",   regex=True)
filtered_df['content'] = filtered_df["content"].replace(r"Physical Index", "",   regex=True)
filtered_df['content'] = filtered_df["content"].replace(r"Fixed Price Swap", "", regex=True)

In [ ]:
# check the time zone

df['timezone'] = df['Date'].apply(lambda x: re.search(r'\((.*?)\)', x).group(1))
df['timezone'].unique()

df.timezone.value_counts()

# Pacific Standard Time (PST) or Pacific Daylight Time (PDT)

In [ ]:
filtered_df.head(2)

In [ ]:
from datetime import datetime
from dateutil import parser, tz

# Define the date format
date_format = "%a, %d %b %Y %H:%M:%S %z (%Z)"

# Define the timezone to convert to
timezone = tz.UTC

# Define a function to convert a date string to a datetime object in UTC timezone
def convert_to_utc(date_string):
    dt_local = parser.parse(date_string)
    dt_utc = dt_local.astimezone(timezone)
    return dt_utc

# Apply the function to the date column
filtered_df["Date_UTC"] = filtered_df["Date"].apply(convert_to_utc)

# Filter the DataFrame to get dates before 2001-05-01
filtered_df = filtered_df[filtered_df["Date_UTC"] < datetime(2001, 5, 1, tzinfo=timezone)]


In [ ]:
filtered_df.tail(10)

# merge with status dataset

In [ ]:
# status.info()

status = pd.read_csv('https://raw.githubusercontent.com/burgersmoke/enron-formality/master/enron_employee_positions/diesner_employee_email_positions.csv', names=["Name", "Email", "Position", "Rank"])
status_wt_na = status.dropna()
print(status.shape, status_wt_na.shape)

# Assuming that "status" is a Pandas DataFrame containing columns "Position" and "Rank"
rank_counts = status['Position'].value_counts(normalize=False).reset_index()
rank_counts.columns = ['Position', 'Num_Employees']

status = status.merge(rank_counts, on='Position', how='left')

# Print the updated "status" DataFrame with new columns
status = status.dropna(subset=["Position", "Rank", "Num_Employees"])
status_unique = status[["Position", "Rank", "Num_Employees"]].drop_duplicates()

status_unique.Num_Employees = status_unique.Num_Employees.round(0).astype(int)

# Print the unique values of Position, Rank, and Num_Employees
status_unique.sort_values(by="Rank", ascending=False)



In [ ]:
import pandas as pd

status = pd.read_csv('https://raw.githubusercontent.com/burgersmoke/enron-formality/master/enron_employee_positions/diesner_employee_email_positions.csv', names=["Name", "Email", "Position", "Rank"])


status = status.dropna(subset=["Position"])
status.Position.value_counts()


print(status.Position.value_counts())

In [ ]:
merged_df = pd.merge(status, filtered_df, left_on='Email',right_on="Sender", how='inner', suffixes=("Sender_", "sender_"))
merged_df = merged_df.rename(columns={'Name':'Sender_Name',
                                      "Email":"Sender_Email",
                                      "Position":"Sender_Position",
                                      "Rank":"Sender_Rank"})

In [ ]:
merged_df = pd.merge(status, merged_df, left_on='Email',right_on="Receiver", how='inner')
merged_df = merged_df.rename(columns={'Name':'Receiver_Name',
                                      "Email":"Receiver_Email",
                                      "Position":"Receiver_Position",
                                      "Rank":"Receiver_Rank"})
merged_df.head()

In [ ]:
merged_df.info()

In [ ]:
merged_df = merged_df.drop(["Sender", "Receiver", "n_receivers", "CC", "Sender_domain", "Receiver_domain"], axis=1)

In [ ]:
merged_df["Sender-Receiver"] = merged_df["Sender_Rank"] - merged_df["Receiver_Rank"]
merged_df.head()

def get_direction(x):
    if x == 0:
        return "same"
    elif x < 0:
        return "upward"
    else:
        return "downward"
    
merged_df["direction"] = merged_df["Sender-Receiver"].apply(get_direction)


In [ ]:
merged_df.info()

merged_df.direction.value_counts()

In [ ]:
merged_df.sample(1)

In [ ]:
merged_df_new = merged_df.drop_duplicates(subset=['Sender_Email', 'Receiver_Email', 'Subject'], keep='first')


In [ ]:
print(df.shape)
print(merged_df.shape)
print(merged_df_new.shape)

In [ ]:
pd.set_option('display.max_colwidth', None)

text = merged_df_new[merged_df_new['Sender-Receiver'] == -6]["content"].sample(1)
print(text.replace('\n', '\\n'))


In [ ]:
merged_df_new.to_csv("Enron_merged_df.csv", index = False)

from google.colab import files
files.download("Enron_merged_df.csv") 

# Tokenization

In [ ]:
# tokenization

import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')

# initialize the Potter stemmer
porter = PorterStemmer()

# define a function to tokenize and stem a text string
def tokenize_and_stem(text):
    tokens = word_tokenize(text.lower())
    stems = [porter.stem(token) for token in tokens]
    return stems

# apply the tokenize_and_stem function to the content column
merged_df_new['tokens'] = merged_df_new['content'].apply(tokenize_and_stem)


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
nltk.download('stopwords')

# initialize the stop words and punctuation
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

# define a function to clean a single token
def clean_token(token):
    # remove punctuation and numbers
    token = ''.join([char for char in token if char not in punctuation and not char.isdigit()])
    # convert to lowercase
    token = token.lower()
    # remove stopwords
    if token not in stop_words:
        return token

# define a function to clean a list of tokens
def clean_tokens(tokens):
    cleaned_tokens = []
    for token in tokens:
        cleaned_token = clean_token(token)
        if cleaned_token:
            cleaned_tokens.append(cleaned_token)
    return cleaned_tokens

# apply the clean_tokens function to the "tokens" column and store the cleaned tokens in a new "cleaned_tokens" column
merged_df_new['cleaned_tokens'] = merged_df_new['tokens'].apply(clean_tokens)
merged_df_new.head()

# Analysis of tokens by direction

In [ ]:
merged_df_new.to_csv("Enron_filtered_dropna.csv", na_rep="", index=False)

In [ ]:
print(merged_df.shape, merged_df_new.shape)

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# print(stop_words)

word_check = "could"
if word_check in stop_words:
    print(word_check, "is stopwords")
else:
    print("No", word_check, "is not in stopwords" )


In [ ]:
print(merged_df.groupby("direction")["tokens"].apply(lambda x: x.apply(len).mean()))

print("\n")

print(merged_df.groupby("direction")["cleaned_tokens"].apply(lambda x: x.apply(len).mean()))


In [ ]:
merged_df_new.info()

In [ ]:
print(merged_df_new.groupby("direction")["tokens"].apply(lambda x: x.apply(len).mean()))

print("\n")

print(merged_df_new.groupby("direction")["cleaned_tokens"].apply(lambda x: x.apply(len).mean()))


In [ ]:
merged_df_new["n_tokens"] = merged_df_new["tokens"].apply(len)
merged_df_new["n_cleaned_tokens"] = merged_df_new["cleaned_tokens"].apply(len)

In [ ]:
merged_df_new.head()

In [ ]:
merged_df_new.groupby("direction")["n_tokens"].mean()

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# fit an ANOVA model with the 'values' column as the dependent variable and the 'group' column as the independent variable
model = ols('n_cleaned_tokens ~ C(direction)', data=merged_df_new).fit()

# perform an ANOVA test
anova_table = sm.stats.anova_lm(model, typ=2)

# print the ANOVA table
print(anova_table)
print("\n\n")

# perform Tukey's HSD test
tukey_results = pairwise_tukeyhsd(merged_df_new['n_cleaned_tokens'], merged_df_new['direction'])

# print the Tukey's HSD results
print(tukey_results)
print("\n\n")

import matplotlib.pyplot as plt
means =merged_df_new.groupby('direction')['n_cleaned_tokens'].mean()
plt.bar(means.index, means.values)
plt.title('Mean values by category')
plt.xlabel('Category')
plt.ylabel('Mean value')
plt.show()


In [ ]:
print(merged_df.groupby("Sender-Receiver")["tokens"].apply(lambda x: x.apply(len).mean()))

print("\n")

print(merged_df.groupby("Sender-Receiver")["cleaned_tokens"].apply(lambda x: x.apply(len).mean()))